Применение Beggs_Brill_class, попытка исправления и приведения в боевую готовность

Построение множества свойств для отладки

In [1]:
import sys
sys.path.append('../')
import uPVT.PVT as PVT
import Beggs_Brill_class as BB

import plotly.plotly as py
import plotly.graph_objs as go

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools

init_notebook_mode(connected=True)



fluid = PVT.FluidStanding()
hydr_cor = BB.Beggs_Brill_cor()

p_initial_mpa = 7
t_initial_c = 90
h_initial_m = 3000
step_m = 10
step_cm = 0.03


PT = BB.PT(p_initial_mpa,t_initial_c)



h_list = [h_initial_m]
p_list = [p_initial_mpa]
t_list = [t_initial_c]
grad_list = [0]
liquid_content_theta = [0]
flow_regime = [-1]
liquid_content = [0]
rhos_list = [0]
rhon_list = [0]
rhol_list = [0]
mu_mix_list = [0]
mu_water_list  = [0]
mu_oil_list = [0]
mu_gas_list = [0]
mu_liquid_list = [0]
for i in range(300):
    if PT.p_mpa > 0:
        
        fluid.calc(PT.p_mpa * 10,PT.T_C)
        hydr_cor.print_all = False
        hydr_cor.mu_gas_pasec = fluid.mu_gas_cP /1000
        hydr_cor.mu_oil_pasec = fluid.mu_oil_cP /1000
        hydr_cor.rho_oil_kgm3 = fluid.rho_oil_kgm3
        hydr_cor.rho_gas_kgm3 = fluid.rho_gas_kgm3
        hydr_cor.gas_rate_on_surface_m3day = hydr_cor.oil_rate_on_surface_m3day * fluid.rsb_m3m3
        hydr_cor.rs_m3m3 = fluid.rs_m3m3
        hydr_cor.bg_m3m3 = fluid.bg_m3m3
        hydr_cor.bo_m3m3 = fluid.bo_m3m3
        grad_pam = hydr_cor.calc_grad(PT)
        
        p = p_list[-1] - step_m * grad_pam / 10**6
        t = t_list[-1] - step_m * step_cm
        h = h_list[-1] - step_m
        grad_list.append(grad_pam/1000)
        liquid_content_theta.append(hydr_cor.volume_liquid_content_with_Pains_cor)
        flow_regime.append(hydr_cor.flow_regime)
        liquid_content.append(hydr_cor.liquid_content)
        rhos_list.append(hydr_cor.rhos_kgm3)
        rhon_list.append(hydr_cor.rhon_kgm3)
        rhol_list.append(hydr_cor.rho_liquid_kgm3)
        mu_mix_list.append(hydr_cor.mu_mix_noslip_pas)
        mu_oil_list.append(hydr_cor.mu_oil_pasec)
        mu_water_list.append(hydr_cor.mu_water_pasec)
        mu_gas_list.append(hydr_cor.mu_gas_pasec)
        mu_liquid_list.append(hydr_cor.mu_liquid_pas)
    if p > 0:
        p_list.append(p)
    else:
        p_list.append(p_list[-1])
    t_list.append(t)
    h_list.append(h)
    PT.p_mpa = p
    PT.T_C = t
    hydr_cor.pressure_pa = p * 10**6
    hydr_cor.temperature_c = t
    

0.04580384554217342
0.04580384554217342


In [2]:
tracep = go.Scattergl(
    x = p_list,
    y = h_list,
    name = 'P_MPa',
    mode = 'markers'
)

tracet = go.Scattergl(
    x = t_list,
    y = h_list,
    name = 'T_C',
    mode = 'markers'
)

tracegrad = go.Scattergl(
    x = grad_list,
    y = h_list,
    name = 'Grad_pam/1000',
    mode = 'markers'
)


data = [tracep, tracet, tracegrad]

layout = dict(title = 'БеггсБрилл снизу вверх, H(P,T)',  yaxis=dict(autorange='reversed'))

fig = dict(data=data, layout=layout)

iplot(fig, filename='basic-scatter')

In [3]:
trace_hltetta = go.Scattergl(
    x = liquid_content_theta,
    y = h_list,
    name = 'hltetta',
    mode = 'markers'
)

trace_flow_regime = go.Scattergl(
    x = flow_regime,
    y = h_list,
    name = 'flow_regime',
    mode = 'markers'
)
trace_liquid_content = go.Scattergl(
    x = liquid_content,
    y = h_list,
    name = 'liquid_content',
    mode = 'markers'
)




data = [trace_hltetta, trace_flow_regime, trace_liquid_content]

layout = dict(title = 'БеггсБрилл снизу вверх, H(P,T)',  yaxis=dict(autorange='reversed'))

fig = dict(data=data, layout=layout)

iplot(fig, filename='basic-scatter')

In [4]:
trace_rhos_list = go.Scattergl(
    x = rhos_list,
    y = h_list,
    name = 'rhos_list',
    mode = 'markers'
)

trace_rhol_list = go.Scattergl(
    x = rhol_list,
    y = h_list,
    name = 'rhol_list',
    mode = 'markers'
)
trace_rhon_list = go.Scattergl(
    x = rhon_list,
    y = h_list,
    name = 'rhon_list',
    mode = 'markers'
)




data = [trace_rhos_list, trace_rhol_list, trace_rhon_list]

layout = dict(title = 'БеггсБрилл снизу вверх, H(P,T)',  yaxis=dict(autorange='reversed'))

fig = dict(data=data, layout=layout)

iplot(fig, filename='basic-scatter')

90

In [5]:
trace_mu_mix_list = go.Scattergl(
    x = mu_mix_list,
    y = h_list,
    name = 'mu_mix_list',
    mode = 'markers'
)

trace_mu_oil_list = go.Scattergl(
    x = mu_oil_list,
    y = h_list,
    name = 'mu_oil_list',
    mode = 'markers'
)
trace_mu_water_list = go.Scattergl(
    x = mu_water_list,
    y = h_list,
    name = 'mu_water_list',
    mode = 'markers'
)

trace_mu_gas_list = go.Scattergl(
    x = mu_gas_list,
    y = h_list,
    name = 'mu_gas_list',
    mode = 'markers'
)
trace_mu_liquid_list = go.Scattergl(
    x = mu_liquid_list,
    y = h_list,
    name = 'mu_liquid_list',
    mode = 'markers'
)




data = [trace_mu_mix_list, trace_mu_oil_list, trace_mu_water_list, trace_mu_gas_list, trace_mu_liquid_list  ]

layout = dict(title = 'БеггсБрилл снизу вверх, H(P,T)',  yaxis=dict(autorange='reversed'))

fig = dict(data=data, layout=layout)

iplot(fig, filename='basic-scatter')
